In [ ]:
import os
import random
import shutil
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_limited_dataset(src_dir, dst_dir, limit_per_class=500):
    src_dir = Path(src_dir)
    dst_dir = Path(dst_dir)
    dst_dir.mkdir(parents=True, exist_ok=True)

    for class_folder in os.listdir(src_dir):
        class_path = src_dir / class_folder
        if not class_path.is_dir():
            continue

        images = os.listdir(class_path)
        random.shuffle(images)
        selected_images = images[:limit_per_class]

        target_class_path = dst_dir / class_folder
        target_class_path.mkdir(parents=True, exist_ok=True)

        for img in selected_images:
            src_img_path = class_path / img
            dst_img_path = target_class_path / img
            shutil.copy(src_img_path, dst_img_path)

    print(f"✅ Done! Limited dataset saved at: {dst_dir}")


# Step 1: Limit dataset to 500/class
src = r"\kaggle\sentiment analysis\archive (1)\images\Data"
dst = r"\kaggle\sentiment analysis\new_dataset"
create_limited_dataset(src, dst, limit_per_class=500)

# Step 2: Load with ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1  # 80% train, 20% val
)

train_gen = datagen.flow_from_directory(
    dst,
    target_size=(48, 48),
    color_mode='rgb',   # <<--- changed
    batch_size=32,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    dst,
    target_size=(48, 48),
    color_mode='rgb',   # <<--- changed
    batch_size=32,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.optimizers import Adam

def build_xception_model(input_shape=(48, 48, 1), num_classes=7, fine_tune=False):
    # Load Xception base model without top classification layers
    base_model = Xception(weights='imagenet', include_top=False, input_tensor=Input(shape=input_shape))

    if not fine_tune:
        # Freeze base model layers
        for layer in base_model.layers:
            layer.trainable = False

    # Add custom top layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x) #max pooling 
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    # Construct the full model
    model = Model(inputs=base_model.input, outputs=predictions)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.0001), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

    return model

c:\Users\surji\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:

# Train the model
history = model.fit(
    train_generator,
    epochs=5,
    validation_data=val_generator
)

c:\Users\surji\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
165/165 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step - accuracy: 0.6022 - loss: 0.9753

c:\Users\surji\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


165/165 ━━━━━━━━━━━━━━━━━━━━ 54s 314ms/step - accuracy: 0.6028 - loss: 0.9738 - val_accuracy: 0.8187 - val_loss: 0.4552
Epoch 2/5
165/165 ━━━━━━━━━━━━━━━━━━━━ 51s 311ms/step - accuracy: 0.7865 - loss: 0.5897 - val_accuracy: 0.8451 - val_loss: 0.3944
Epoch 3/5
165/165 ━━━━━━━━━━━━━━━━━━━━ 59s 355ms/step - accuracy: 0.8292 - loss: 0.4461 - val_accuracy: 0.8530 - val_loss: 0.3718
Epoch 4/5
165/165 ━━━━━━━━━━━━━━━━━━━━ 50s 305ms/step - accuracy: 0.8280 - loss: 0.4551 - val_accuracy: 0.8574 - val_loss: 0.3779
Epoch 5/5
165/165 ━━━━━━━━━━━━━━━━━━━━ 50s 305ms/step - accuracy: 0.8565 - loss: 0.3953 - val_accuracy: 0.8759 - val_loss: 0.3199


In [6]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc*100:.2f}%")

 1/36 ━━━━━━━━━━━━━━━━━━━━ 3s 91ms/step - accuracy: 0.9062 - loss: 0.1879

c:\Users\surji\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - accuracy: 0.8875 - loss: 0.2901
Test Accuracy: 88.40%


In [ ]:
model.save('result.h5')


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# Path to the image you want to predict
image_path = r""  # Replace with your image path

# Load the image and resize it to (150, 150)
img = image.load_img(image_path, target_size=(150, 150))  # Resize to match the model's input size

# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Add an extra dimension to the array to simulate a batch (1, 150, 150, 3)
img_array = np.expand_dims(img_array, axis=0)

# Normalize the image array (since the model was trained on rescaled images)
img_array = img_array / 255.0

# Load the trained model
model = tf.keras.models.load_model(r"")  # Load your trained model

# Make the prediction
predictions = model.predict(img_array)

# Get the index of the class with the highest predicted probability
predicted_class_index = np.argmax(predictions, axis=1)[0]

# Print the predicted class index
print(f"Predicted class index: {predicted_class_index}")


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 36992, but received input with shape (1, 86528)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 224, 224, 3), dtype=float32)
  • training=False
  • mask=None